In [3]:
import matplotlib.pyplot as plt
import numpy as np
import shared_data as sd
import utility_functions as uf
from config_file_utils import ConfigFile
import os
from glob import glob
import pandas as pd
from collections import defaultdict
import seaborn as sns

In [5]:
accuracy_mean_dict = defaultdict(dict)
annot_dict = defaultdict(dict)
for config_file_name in os.listdir(sd.CONFIG_FILES_BEST_DIR):
    cf = ConfigFile(f"{sd.CONFIG_FILES_DIR}/{config_file_name}")
    y = uf.get_y(
        sd.PARTITION_N_TRJS,
        sd.TRJ_LEN,
        len(sd.PAIRS[cf.pair_name]),
        cf.config_dict
    )
    y_predicted = np.load(glob(f"*validation_{cf.config_index}"))
    accuracy_mean, accuracy_sem = uf.get_accuracy_mean_and_sem(y_predicted, y, cf, 2)
    pipeline_name = f"{cf.dim_reduction_name}/{cf.model_name}"
    accuracy_mean_dict[cf.pair_name][pipeline_name][cf.sel_name] = accuracy_mean
    annot_dict[cf.pair_name][pipeline_name][cf.sel_name] = f"{accuracy_mean:.2f}\n\u00B1\n{accuracy_sem:.2f}"
for pair_name in sd.PAIRS:
    accuracy_mean_df = pd.DataFrame(accuracy_mean_dict[pair_name])
    annot_df = pd.DataFrame(annot_dict[pair_name])
    plt.figure(figsize=(20, 9))
    plt.title(f"Classification model comparison for {pair_name}")
    sns.heatmap(
        X=accuracy_mean_df, 
        vmin=0.5, 
        vmax=1, 
        cmap="RdBu", 
        annot=annot_df,
        fmt="",
        xticklabels=list(accuracy_mean_df.index),
        yticklabels=list(accuracy_mean_df.columns),
        cbar_kws={"label": "Cross validation accuracy", "pad": 0.01},
    )
    plt.xticks(rotation=15)

TypeError: expected str, bytes or os.PathLike object, not list